In [1]:
import os
import rasterio as rio
import numpy as np
import shapely
import pyproj
import geopandas as gpd
import matplotlib.pyplot as plt
import rioxarray as riox
import rasterio as rio
import xarray as xr
import netCDF4
from osgeo import gdal
import pandas as pd
from datetime import datetime
import dask.array

import sys
sys.path.append('../')
import snowFun

In [2]:
# define folder and file paths
folder_AGVA = os.path.join('C:',os.sep,'Users','lzell','OneDrive - Colostate','Desktop',"AGVA")
folder_dems = os.path.join(folder_AGVA, "DEMs", "time_varying_DEMs", "10m")
folder_class = os.path.join(folder_AGVA, 'classified images', 'S2_Classified_Cloudmasked_Merged')
folder_cloud = os.path.join(folder_AGVA, 'classified images', 'S2_Cloud_Merged')
folder_meta = os.path.join(folder_AGVA, "classified images", "meta csv", "S2")
folder_mask = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Masks')

# open rgi
path_rgi = os.path.join(folder_AGVA, 'RGI', "rgi_2km_o3regions", "rgi_2km_o3regions.shp")
rgi_gdf = gpd.read_file(path_rgi, drop='geometry')

In [3]:
### choose if you want to do only the 45 validation glaciers
validation_only = 0

# select which rgis to analyze
if validation_only:
    folder_save = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Validation')
    
else:
    folder_save = os.path.join(folder_AGVA, 'Derived products', 'S2')

# get list of rgis to analyze
# rgis_to_analyze = list(set( [ i[3:17] for i in os.listdir(os.path.join(folder_save, "Average AAs")) if i[-3:]=='tif' ] ))

# # get list of glacier area for each rgi
# areas = [rgi_gdf[rgi_gdf['RGIId']==i]['Area'].values for i in rgis_to_analyze]

# # make df
# rgis_to_analyze_df = pd.DataFrame({"RGIId":rgis_to_analyze, 'Area':areas})

# # sort however you want
# rgis_to_analyze_df = rgis_to_analyze_df.sort_values('Area')

# # grab rgi names
# rgis_to_analyze = rgis_to_analyze_df['RGIId'].values

# print(len(rgis_to_analyze_df))
# # print(rgis_to_analyze[:10])
# # print(rgis_to_analyze_df[:10])

### First step: open glacier DEM and calculate hypsometric ELA given observed AAR, hypsometric AAR given ELA
#### Then calculate the AAR if we raised ELA 10m, 20m, 30m, etc...

In [ ]:
# open final aar/ela df
df_path = os.path.join(folder_save, 'Average AAs', 'final_glacier_stats.csv')
df_obs = pd.read_csv(df_path)

# get rgi list
rgis_to_analyze = df_obs['RGIId'].values

# initiate list to hold outputs
outputs = []

for i in range(len(rgis_to_analyze)):
    
    # subset rgi to single outline, by choosing rgiid or rgi name
    rgiid = rgis_to_analyze[i]
    
#     if rgiid!="RGI60-01.09162": continue

    # quickly grab glacier area
    ga = rgi_gdf[rgi_gdf['RGIId']==rgiid]['Area'].values[0]

    # print progress
    print(f"Starting {i+1} of {len(rgis_to_analyze)}: {rgiid}  {ga} km2")
    
    # grab just this rgi geometry and info
    rgi_single = rgi_gdf[rgi_gdf['RGIId']==rgiid].to_crs("EPSG:3338")
    single_geometry = rgi_single.geometry
    single_obs = df_obs[df_obs['RGIId']==rgiid]
    
    # define the coarsen scale
#     if ga>3000:
#         scale=5
    if ga>1000:
        scale=5
    elif ga>500:
        scale=3
    else:
        scale=1
        
    # open glacier mask
    glacier_mask = xr.open_dataset(os.path.join(folder_mask, f"S2_{rgiid}_mask.nc"), chunks='auto').glacier
    
    if ga>500:
        glacier_mask = glacier_mask.coarsen({"x":scale, "y":scale}, boundary="trim").median(skipna=True).astype('uint8')  

    # open dem for year 2020
    xr_dem = snowFun.get_year_DEM(single_geometry, 2020, smoothed=0) 
    
    # coarsen dem
    if ga>500:
        xr_dem = xr_dem.sel({"x":glacier_mask.x, "y":glacier_mask.y})

    # shave off edges to make sure dem, mask match
    xr_dem = xr_dem.sel(x=slice( min(glacier_mask.x.values), max(glacier_mask.x.values) ), y=slice(max(glacier_mask.y.values),min(glacier_mask.y.values)))
    glacier_mask = glacier_mask.sel(x=slice( min(xr_dem.x.values), max(xr_dem.x.values) ), y=slice(max(xr_dem.y.values),min(xr_dem.y.values)))

    # flatten dem values into single list, removing 0s
    dem_values = np.ravel(xr_dem.values)
    dem_values = dem_values[ (dem_values!=0) & ~np.isnan(dem_values)]
    dem_length = len(dem_values)
    
    # get the observed aar, ela
    obs_ela = single_obs['ela'].values[0]
    obs_aar = single_obs['aar'].values[0]
    
    # define function that will calculate the expected AAR of a given ELA
    def get_hyps_aar(ela_input):
        return round( np.nansum( (dem_values>=ela_input) )/dem_length, 4)
    
    # calculate hypsometric aar, ela
    hyps_ela = np.nanpercentile(dem_values, (1-obs_aar)*100)
    hyps_aar = get_hyps_aar(obs_ela)
    
    # calculate hypsometric AAR if we raise ela by 100m
    hyps_aar_20 = get_hyps_aar(obs_ela+20)
    hyps_aar_40 = get_hyps_aar(obs_ela+40)
    hyps_aar_60 = get_hyps_aar(obs_ela+60)
    hyps_aar_80 = get_hyps_aar(obs_ela+80)
    hyps_aar_100 = get_hyps_aar(obs_ela+100)
    hyps_aar_120 = get_hyps_aar(obs_ela+120)
    hyps_aar_140 = get_hyps_aar(obs_ela+140)
    hyps_aar_160 = get_hyps_aar(obs_ela+160)
    hyps_aar_180 = get_hyps_aar(obs_ela+180)
    hyps_aar_200 = get_hyps_aar(obs_ela+200)
    
    # create dictionary to save
    out_dic = {"RGIId":rgiid, "obs_ela":obs_ela, "obs_aar":obs_aar,
               "hyps_ela":hyps_ela, "hyps_aar":hyps_aar,
               "hyps_aar_20":hyps_aar_20, "hyps_aar_40":hyps_aar_40, "hyps_aar_60":hyps_aar_60, "hyps_aar_80":hyps_aar_80, "hyps_aar_100":hyps_aar_100,
               "hyps_aar_120":hyps_aar_120, "hyps_aar_140":hyps_aar_140, "hyps_aar_160":hyps_aar_160, "hyps_aar_180":hyps_aar_180, "hyps_aar_200":hyps_aar_200}
  
    # save to list
    outputs.append(out_dic)

print('Done!')

# format to df, save
out_df = pd.DataFrame(outputs)
out_path = os.path.join(folder_save, 'Average AAs', 'hyspometric_stats.csv')
out_df.to_csv(out_path, index=False)

In [ ]:
print(f"Given observed ELA of: {obs_ela}, we expect AAR of: {hyps_aar}")
print(f"Given observed AAR of: {obs_aar}, we expect ELA of: {hyps_ela}")
print(f"If ELA rises by 100 meters, the hypsometric AAR will be: {hyps_aar_100}")
print(f"If ELA rises by 200 meters, the hypsometric AAR will be: {hyps_aar_200}")

In [ ]:
out_df

In [ ]:
xr_dem

In [ ]:
df_obs

### Second step: go through those snow cover fraction products to calculate the ELA